In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [4]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [5]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

In [6]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0370, g_loss: 4.0494, D(x): 1.00, D(G(z)): 0.03
Epoch [0/200], Step [400/600], d_loss: 0.0963, g_loss: 5.6307, D(x): 0.98, D(G(z)): 0.06
Epoch [0/200], Step [600/600], d_loss: 0.0529, g_loss: 4.7905, D(x): 0.99, D(G(z)): 0.04
Epoch [1/200], Step [200/600], d_loss: 0.0992, g_loss: 3.7186, D(x): 0.97, D(G(z)): 0.06
Epoch [1/200], Step [400/600], d_loss: 0.4846, g_loss: 2.6830, D(x): 0.87, D(G(z)): 0.17
Epoch [1/200], Step [600/600], d_loss: 0.3593, g_loss: 3.9742, D(x): 0.92, D(G(z)): 0.13
Epoch [2/200], Step [200/600], d_loss: 0.1610, g_loss: 3.8998, D(x): 0.94, D(G(z)): 0.08
Epoch [2/200], Step [400/600], d_loss: 1.6981, g_loss: 1.9047, D(x): 0.51, D(G(z)): 0.21
Epoch [2/200], Step [600/600], d_loss: 0.4577, g_loss: 4.0449, D(x): 0.88, D(G(z)): 0.15
Epoch [3/200], Step [200/600], d_loss: 1.0501, g_loss: 1.9021, D(x): 0.73, D(G(z)): 0.36
Epoch [3/200], Step [400/600], d_loss: 0.3423, g_loss: 2.9933, D(x): 0.91, D(G(z)): 0.14
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.3566, g_loss: 3.7376, D(x): 0.91, D(G(z)): 0.12
Epoch [31/200], Step [200/600], d_loss: 0.3555, g_loss: 5.2485, D(x): 0.85, D(G(z)): 0.04
Epoch [31/200], Step [400/600], d_loss: 0.5124, g_loss: 3.3673, D(x): 0.89, D(G(z)): 0.19
Epoch [31/200], Step [600/600], d_loss: 0.3138, g_loss: 3.8285, D(x): 0.93, D(G(z)): 0.12
Epoch [32/200], Step [200/600], d_loss: 0.6737, g_loss: 3.7291, D(x): 0.93, D(G(z)): 0.29
Epoch [32/200], Step [400/600], d_loss: 0.3601, g_loss: 4.1837, D(x): 0.92, D(G(z)): 0.16
Epoch [32/200], Step [600/600], d_loss: 0.3096, g_loss: 5.4609, D(x): 0.89, D(G(z)): 0.05
Epoch [33/200], Step [200/600], d_loss: 0.3840, g_loss: 3.9842, D(x): 0.88, D(G(z)): 0.11
Epoch [33/200], Step [400/600], d_loss: 0.4933, g_loss: 4.4423, D(x): 0.95, D(G(z)): 0.23
Epoch [33/200], Step [600/600], d_loss: 0.3592, g_loss: 5.0640, D(x): 0.86, D(G(z)): 0.06
Epoch [34/200], Step [200/600], d_loss: 0.2794, g_loss: 4.3606, D(x): 0.89, D(G(z)): 0.04
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.6021, g_loss: 2.1048, D(x): 0.80, D(G(z)): 0.19
Epoch [61/200], Step [600/600], d_loss: 0.7725, g_loss: 2.9980, D(x): 0.68, D(G(z)): 0.12
Epoch [62/200], Step [200/600], d_loss: 0.5666, g_loss: 1.9507, D(x): 0.83, D(G(z)): 0.21
Epoch [62/200], Step [400/600], d_loss: 0.6087, g_loss: 2.3087, D(x): 0.83, D(G(z)): 0.24
Epoch [62/200], Step [600/600], d_loss: 0.7304, g_loss: 2.5738, D(x): 0.84, D(G(z)): 0.30
Epoch [63/200], Step [200/600], d_loss: 0.6194, g_loss: 2.3514, D(x): 0.80, D(G(z)): 0.20
Epoch [63/200], Step [400/600], d_loss: 0.8458, g_loss: 2.1725, D(x): 0.68, D(G(z)): 0.17
Epoch [63/200], Step [600/600], d_loss: 0.6589, g_loss: 2.7265, D(x): 0.84, D(G(z)): 0.29
Epoch [64/200], Step [200/600], d_loss: 1.0543, g_loss: 2.2697, D(x): 0.61, D(G(z)): 0.18
Epoch [64/200], Step [400/600], d_loss: 0.5641, g_loss: 1.9358, D(x): 0.85, D(G(z)): 0.22
Epoch [64/200], Step [600/600], d_loss: 0.6510, g_loss: 2.2224, D(x): 0.80, D(G(z)): 0.25
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.6045, g_loss: 1.9058, D(x): 0.79, D(G(z)): 0.20
Epoch [92/200], Step [400/600], d_loss: 1.0016, g_loss: 1.6326, D(x): 0.69, D(G(z)): 0.26
Epoch [92/200], Step [600/600], d_loss: 0.8268, g_loss: 1.7594, D(x): 0.73, D(G(z)): 0.25
Epoch [93/200], Step [200/600], d_loss: 0.9958, g_loss: 1.7751, D(x): 0.74, D(G(z)): 0.35
Epoch [93/200], Step [400/600], d_loss: 0.5800, g_loss: 2.6969, D(x): 0.84, D(G(z)): 0.23
Epoch [93/200], Step [600/600], d_loss: 0.6908, g_loss: 1.9294, D(x): 0.85, D(G(z)): 0.28
Epoch [94/200], Step [200/600], d_loss: 0.7146, g_loss: 1.9196, D(x): 0.81, D(G(z)): 0.31
Epoch [94/200], Step [400/600], d_loss: 0.7072, g_loss: 2.4683, D(x): 0.73, D(G(z)): 0.20
Epoch [94/200], Step [600/600], d_loss: 0.7414, g_loss: 2.3873, D(x): 0.72, D(G(z)): 0.21
Epoch [95/200], Step [200/600], d_loss: 0.6691, g_loss: 2.0786, D(x): 0.83, D(G(z)): 0.29
Epoch [95/200], Step [400/600], d_loss: 0.8550, g_loss: 1.5331, D(x): 0.79, D(G(z)): 0.32
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8090, g_loss: 1.5360, D(x): 0.69, D(G(z)): 0.25
Epoch [122/200], Step [600/600], d_loss: 0.8847, g_loss: 1.8632, D(x): 0.68, D(G(z)): 0.24
Epoch [123/200], Step [200/600], d_loss: 0.8524, g_loss: 2.0299, D(x): 0.68, D(G(z)): 0.22
Epoch [123/200], Step [400/600], d_loss: 0.8970, g_loss: 1.5242, D(x): 0.70, D(G(z)): 0.28
Epoch [123/200], Step [600/600], d_loss: 0.8739, g_loss: 1.3553, D(x): 0.73, D(G(z)): 0.29
Epoch [124/200], Step [200/600], d_loss: 1.0241, g_loss: 1.8157, D(x): 0.71, D(G(z)): 0.34
Epoch [124/200], Step [400/600], d_loss: 0.8344, g_loss: 2.3925, D(x): 0.73, D(G(z)): 0.27
Epoch [124/200], Step [600/600], d_loss: 0.8645, g_loss: 1.8932, D(x): 0.69, D(G(z)): 0.25
Epoch [125/200], Step [200/600], d_loss: 0.9030, g_loss: 2.0078, D(x): 0.63, D(G(z)): 0.17
Epoch [125/200], Step [400/600], d_loss: 1.0106, g_loss: 2.0310, D(x): 0.72, D(G(z)): 0.38
Epoch [125/200], Step [600/600], d_loss: 0.7893, g_loss: 1.4759, D(x): 0.71, D(G(z)): 0.24

Epoch [152/200], Step [600/600], d_loss: 1.0436, g_loss: 1.7200, D(x): 0.67, D(G(z)): 0.30
Epoch [153/200], Step [200/600], d_loss: 0.9338, g_loss: 1.5683, D(x): 0.87, D(G(z)): 0.45
Epoch [153/200], Step [400/600], d_loss: 0.8138, g_loss: 1.7303, D(x): 0.70, D(G(z)): 0.25
Epoch [153/200], Step [600/600], d_loss: 1.0449, g_loss: 1.6331, D(x): 0.72, D(G(z)): 0.36
Epoch [154/200], Step [200/600], d_loss: 0.9039, g_loss: 1.4658, D(x): 0.75, D(G(z)): 0.33
Epoch [154/200], Step [400/600], d_loss: 0.9932, g_loss: 1.4798, D(x): 0.77, D(G(z)): 0.41
Epoch [154/200], Step [600/600], d_loss: 0.9863, g_loss: 1.7786, D(x): 0.69, D(G(z)): 0.32
Epoch [155/200], Step [200/600], d_loss: 0.8113, g_loss: 1.5887, D(x): 0.76, D(G(z)): 0.34
Epoch [155/200], Step [400/600], d_loss: 0.8706, g_loss: 1.4593, D(x): 0.72, D(G(z)): 0.30
Epoch [155/200], Step [600/600], d_loss: 0.8708, g_loss: 1.9049, D(x): 0.70, D(G(z)): 0.27
Epoch [156/200], Step [200/600], d_loss: 1.0215, g_loss: 0.9707, D(x): 0.69, D(G(z)): 0.36

Epoch [183/200], Step [200/600], d_loss: 0.7524, g_loss: 2.0708, D(x): 0.73, D(G(z)): 0.23
Epoch [183/200], Step [400/600], d_loss: 0.9666, g_loss: 1.7592, D(x): 0.64, D(G(z)): 0.25
Epoch [183/200], Step [600/600], d_loss: 1.0063, g_loss: 1.6668, D(x): 0.65, D(G(z)): 0.32
Epoch [184/200], Step [200/600], d_loss: 0.8716, g_loss: 1.7609, D(x): 0.74, D(G(z)): 0.31
Epoch [184/200], Step [400/600], d_loss: 1.1673, g_loss: 1.4707, D(x): 0.55, D(G(z)): 0.26
Epoch [184/200], Step [600/600], d_loss: 0.9600, g_loss: 1.8343, D(x): 0.71, D(G(z)): 0.29
Epoch [185/200], Step [200/600], d_loss: 1.1128, g_loss: 1.4253, D(x): 0.66, D(G(z)): 0.35
Epoch [185/200], Step [400/600], d_loss: 0.9021, g_loss: 1.1939, D(x): 0.77, D(G(z)): 0.39
Epoch [185/200], Step [600/600], d_loss: 0.8582, g_loss: 1.4981, D(x): 0.74, D(G(z)): 0.33
Epoch [186/200], Step [200/600], d_loss: 0.9000, g_loss: 1.1946, D(x): 0.71, D(G(z)): 0.31
Epoch [186/200], Step [400/600], d_loss: 0.9792, g_loss: 1.3533, D(x): 0.77, D(G(z)): 0.43